In [ ]:
import pandas as pd
from sqlalchemy import create_engine,MetaData
import uuid
from sqlalchemy.dialects.postgresql import UUID 
import sqlalchemy
from airfare_packages import airfare_etl, airfare_classes 
f

In [ ]:
engine = create_engine()


In [3]:
raw_airfare_data = pd.read_csv(r"data/Scraped_dataset.csv")

In [8]:
raw_airfare_data

,Date of Booking,Date of Journey,Airline-Class,Departure Time,Arrival Time,Duration,Total Stops,Price
0,15/01/2023,16/01/2023,SpiceJet \nSG-8169\nECONOMY,20:00\nDelhi,22:05\nMumbai,02h 05m,non-stop,"5,335"
1,15/01/2023,16/01/2023,Indigo \n6E-2519\nECONOMY,23:00\nDelhi,01:20\nMumbai,02h 20m,non-stop,"5,899"
2,15/01/2023,16/01/2023,GO FIRST \nG8- 354\nECONOMY,22:30\nDelhi,00:40\nMumbai,02h 10m,non-stop,"5,801"
3,15/01/2023,16/01/2023,SpiceJet \nSG-8709\nECONOMY,18:50\nDelhi,20:55\nMumbai,02h 05m,non-stop,"5,794"
4,15/01/2023,16/01/2023,Air India \nAI-805\nECONOMY,20:00\nDelhi,22:10\nMumbai,02h 10m,non-stop,"5,955"
...,...,...,...,...,...,...,...,...
452083,15/01/2023,06/03/2023,Vistara \nUK-926\nBUSINESS,06:45\nAhmedabad,19:50\nChennai,13h 05m,1-stop,"65,028"
452084,15/01/2023,06/03/2023,Vistara \nUK-918\nBUSINESS,05:05\nAhmedabad,16:20\nChennai,11h 15m,1-stop,"69,254"
452085,15/01/2023,06/03/2023,Vistara \nUK-918\nBUSINESS,05:05\nAhmedabad,16:20\nChennai,11h 15m,1-stop,"69,254"
452086,15/01/2023,06/03/2023,Vistara \nUK-946\nBUSINESS,08:40\nAhmedabad,19:50\nChennai,11h 10m,1-stop,"72,980"


In [7]:
raw_airfare_data.head(2).columns

Index(['Date of Booking', 'Date of Journey', 'Airline-Class', 'Departure Time',
       'Arrival Time', 'Duration', 'Total Stops', 'Price'],
      dtype='object')

In [4]:
clean_airfare_data = airfare_etl.process_raw_data(raw_airfare_data)

In [5]:
clean_airfare_data.shape

(452088, 13)

In [11]:
db_col_definitions = [ 
    sqlalchemy.Column("id", UUID(as_uuid=True), primary_key=True, default=uuid.uuid4),
    sqlalchemy.Column('date_of_booking', sqlalchemy.Date),  
    sqlalchemy.Column('date_of_journey', sqlalchemy.Date),                    
    sqlalchemy.Column('airline', sqlalchemy.String),
    sqlalchemy.Column('flight_code', sqlalchemy.String),
    sqlalchemy.Column("class", sqlalchemy.String),
    sqlalchemy.Column("connections", sqlalchemy.Integer),
    sqlalchemy.Column("duration_minutes", sqlalchemy.Integer),
    sqlalchemy.Column("departure_time", sqlalchemy.Time),
    sqlalchemy.Column("source_city", sqlalchemy.String),
    sqlalchemy.Column("arrival_time", sqlalchemy.Time),
    sqlalchemy.Column("destination_city", sqlalchemy.String),
    sqlalchemy.Column("price", sqlalchemy.Float)]

In [ ]:
with airfare_classes.db_table("airfare_data",engine,MetaData()) as table:
    table.create_table(db_col_definitions)
    table.initial_data_upload(clean_airfare_data)